# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%pip install datasets causal_conv1d

# Lab

In [14]:
!python Trainer.py

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.
Some weights of the model checkpoint at state-spaces/mamba-130m-hf were not used when initializing MambaForCausalLM: ['backbone.layers.10.mixer.A_log', 'backbone.layers.10.mixer.D', 'backbone.layers.10.mixer.conv1d.bias', 'backbone.layers.10.mixer.conv1d.weight', 'backbone.layers.10.mixer.dt_proj.bias', 'backbone.layers.10.mixer.dt_proj.weight', 'backbone.layers.10.mixer.in_proj.weight', 'backbone.layers.10.mixer.out_proj.weight', 'backbone.layers.10.mixer.x_proj.weight', 'backbone.layers.10.norm.weight', 'backbone.layers.11.mixer.A_log', 'backbone.layers.11.m